# Assigns scenic value

In [4]:
import asyncio
import google.generativeai as genai
import os
import json
import pickle
import math
import numpy as np
from tqdm.asyncio import tqdm_asyncio
from pydantic import BaseModel
from time import perf_counter, sleep
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel("gemini-1.5-flash-8b")
        
class ScenicValue(BaseModel):
    scenic_value: float

async def assign_scenic_description(stop_name, season="summer"):
    result = await model.generate_content_async(
        contents=[
            "Respond using a list.",
            f"Instructions: '{system_prompt1}'",
            f"Context: The stop name is '{stop_name}'. The current season is '{season}'."
        ],
        generation_config=genai.GenerationConfig(
            max_output_tokens=500,
            temperature=0,
        )
    )
    description = result.text
    return description

async def assign_scenic_value(description):
    result = await model.generate_content_async(
        contents=[
            "Respond with one number with one decimal.", 
            f"Instructions: {system_prompt2}", 
            f"Context: {description}"
        ],
        generation_config=genai.GenerationConfig(
            response_mime_type='application/json',
            response_schema=ScenicValue,
            max_output_tokens=50,
            temperature=0,
        )
    )
    scenic_value = json.loads(result.text)["scenic_value"]
    return scenic_value

async def evaluate(stop_name):
    description = await assign_scenic_description(stop_name)
    scenic_value = await assign_scenic_value(description)
    return {stop_name: scenic_value}

async def dummy_evaluate(stop_name):
    scenic_value = np.random.randint(0, 10)
    return {stop_name: scenic_value}

def split_list_into_chunks(original_list, max_chunk_size=2000):
    length = len(original_list)
    num_chunks = math.ceil(length / max_chunk_size)
    base_chunk_size = length // num_chunks
    remainder = length % num_chunks

    chunks = []
    start = 0
    for i in range(num_chunks):
        end = start + base_chunk_size + (1 if i < remainder else 0)
        chunks.append(original_list[start:end])
        start = end

    return chunks

with open('graph_with_coordinates_and_neighbors.pickle', 'rb') as file:
    graph = pickle.load(file)

def sub_graph(data, n=2):
    keys = list(data.keys())
    sliced_keys = keys[:n]
    return {key: data[key] for key in sliced_keys}

graph = sub_graph(graph, n=len(graph)) # set n = len(data) for the full dataset
stop_names = list(graph.keys()) # A list of stops represented as strings
stop_name_chunks = split_list_into_chunks(stop_names)
stop_scenic_values = []

# Creates a list of dictionaries [{stop_name: scenic_value}]
for stop_name_chunk in stop_name_chunks:
    tasks = [evaluate(stop_name) for stop_name in stop_name_chunk]
    stop_scenic_values_chunk = await tqdm_asyncio.gather(*tasks)
    stop_scenic_values += stop_scenic_values_chunk
    # Saving
    with open('stop_scenic_values.json', 'w') as file:
        json.dump(stop_scenic_values, file)
    sleep(60)

# Mapping the scenic values to each stops and its corresponding neighbors
scenic_values_dict = {}
for scenic_value in stop_scenic_values:
    for stop, value in scenic_value.items():
        scenic_values_dict[stop] = value

for stop, info in graph.items():
    if stop in scenic_values_dict:
        info['scenic_value'] = scenic_values_dict[stop]
    for neighbor, neighbor_info in info.get('neighbors', {}).items():
        if neighbor in scenic_values_dict:
            neighbor_info['scenic_value'] = scenic_values_dict[neighbor]

print(graph["Campus Roslagen"])
with open('graph_with_scenic_values.pickle', 'wb') as file:
    pickle.dump(graph, file)

100%|██████████| 1431/1431 [00:00<00:00, 46979.46it/s]


{'neighbors': {'Gustavslund': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 7}, 'Astrid Lindgrens gata': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 4}, 'Backtorp': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 5}, 'Norrtälje busstation': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 2}, 'Malsta vägskäl': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 6}, 'Stockholmsvägen': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 5}, 'Södra Lohärad': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 4}, 'Rösa trafikplats': {'longitude': 18.685677, 'latitude': 59.748096, 'transport_mode': 'bus', 'scenic_value': 2}}, 'latitude': 59.748096, 'longitude': 18.685677, 'scenic_value': 5}


# Assigns scenic value to missing stops

In [2]:
system_prompt1 = """Evaluate the scenic and visual characteristics of public transport stops in Stockholm based on their name, season, surroundings, and notable features.

Instructions:

	1.	Describe surroundings and features with balanced, factual observations. Highlight both positives and negatives constructively. Avoid scoring or biased language.
	2.	Focus on:
	•	Neighborhood/District: Characterize the area’s vibe (e.g., residential, historic) and its scenic impact.
	•	Surroundings: Mention landmarks, natural features, urban cleanliness, safety, or neglect.
	•	Accessibility: Highlight features like signage, seating, and wheelchair access.
	•	Cultural/Historical Context: Note any relevant importance.
	•	Public Perception: Assess if it’s popular among locals or tourists or isolated.
	•	Comparative Context: Describe how it stands out or falls short compared to nearby stops.
	3.	Include negative features explicitly (e.g., industrial surroundings, poor lighting, unsafe areas).
	4.	If data is missing (e.g., no landmarks), state it clearly without guessing.
	5.	Ensure descriptions are consistent across transport types and avoid redundancy.
	6.	Consider seasonal variations (e.g., winter vs. summer scenery).
	7.	Evaluate the stop’s appeal to tourists and explorers based on access, surroundings, and proximity to attractions.
"""

system_prompt2 = """You are tasked with evaluating the scenic value of a public transport stop in Stockholm based on a detailed description. Assign a score from 0 to 10 by considering the following:
	1.	Key Elements:
	•	Stop Name: Identify the stop being evaluated.
	•	Season: Assess how the current season impacts scenic appeal.
	•	Surroundings: Note nearby landmarks, nature, or urban features.
	•	Accessibility: Consider ease of access and its influence on appeal.
	2.	Scenic Value Scale:
	•	0: Extremely unattractive (e.g., industrial, poorly maintained areas).
	•	5: Average appeal (functional but unremarkable).
	•	10: Iconic or uniquely scenic (e.g., waterfront views, exceptional design).
	3.	Balancing Factors:
	•	Positive Features: Highlight views, pleasant surroundings, or unique attributes.
	•	Negative Factors: Prioritize significant drawbacks (e.g., poor maintenance) in scoring.
	•	Seasonality: While seasonal changes matter, water stops should generally score higher.
	4.	Consistency: Be realistic and critical when negative elements dominate.

The final score should reflect a balanced, nuanced judgment of the stop’s scenic appeal.

### Example Input Description:
The stop name is 'Arlanda'. It is located in an airport complex, surrounded by large parking lots and commercial buildings. The surroundings
feel quite sterile, with little to no greenery. During the winter, the area is gray and cold, and there is not much to see other than the
airport itself.

### Example Output:
- **Scenic Value**: 3

### Explanation:
- The description highlights an **unattractive environment** with **industrial surroundings**, lack of greenery, and a **sterile** atmosphere.
Winter's grayness and cold further detract from the visual appeal, leading to a low score.

### Example Input Description:
The stop name is 'Solna Centrum'. The station is located in a modern urban setting with a lot of open space, several nearby cafés, and some
green patches. It feels clean and well-maintained, and during the spring, flowers bloom around the area.

### Example Output:
- **Scenic Value**: 7

### Explanation:
- The description mentions **modern urban features** with **open space**, **cafés**, and **green patches**. The **seasonal appeal** of spring
adds to the overall scenic experience, but the setting isn't particularly striking or unique. Still, it is **clean, well-maintained**, and has
some pleasant features, earning a mid-to-high score.
"""

In [72]:
import asyncio
import google.generativeai as genai
import os
import json
import pickle
import math
import numpy as np
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio
from pydantic import BaseModel
from time import perf_counter, sleep
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY_PAID"])
model = genai.GenerativeModel("gemini-1.5-flash-8b")
        
class ScenicValue(BaseModel):
    scenic_value: float

async def assign_scenic_description(stop_name, season="summer"):
    result = await model.generate_content_async(
        contents=[
            "Respond using a list.",
            f"Instructions: '{system_prompt1}'",
            f"Context: The stop name is '{stop_name}'. The current season is '{season}'."
        ],
        generation_config=genai.GenerationConfig(
            max_output_tokens=500,
            temperature=0,
        )
    )
    description = result.text
    return description

async def assign_scenic_value(description):
    result = await model.generate_content_async(
        contents=[
            "Respond with one number with one decimal.", 
            f"Instructions: {system_prompt2}", 
            f"Context: {description}"
        ],
        generation_config=genai.GenerationConfig(
            response_mime_type='application/json',
            response_schema=ScenicValue,
            max_output_tokens=50,
            temperature=0,
        )
    )
    scenic_value = json.loads(result.text)["scenic_value"]
    return scenic_value

async def evaluate(stop_name):
    description = await assign_scenic_description(stop_name)
    scenic_value = await assign_scenic_value(description)
    return {stop_name: scenic_value}

async def dummy_evaluate(stop_name):
    scenic_value = np.random.randint(0, 10)
    return {stop_name: scenic_value}

def split_list_into_chunks(original_list, max_chunk_size=2000):
    length = len(original_list)
    num_chunks = math.ceil(length / max_chunk_size)
    base_chunk_size = length // num_chunks
    remainder = length % num_chunks

    chunks = []
    start = 0
    for i in range(num_chunks):
        end = start + base_chunk_size + (1 if i < remainder else 0)
        chunks.append(original_list[start:end])
        start = end

    return chunks

with open("graph_with_scenic_values.pickle", "rb") as file:
    graph = pickle.load(file)

# stops with missing scenic values
stop_names = [stop for stop, attributes in graph.items() if attributes.get("scenic_value") is None]
stop_name_chunks = split_list_into_chunks(stop_names, 2000)
stop_scenic_values = []

# Creates a list of dictionaries [{stop_name: scenic_value}]
for stop_name_chunk in tqdm(stop_name_chunks):
    start = perf_counter()
    tasks = [evaluate(stop_name) for stop_name in stop_name_chunk]
    stop_scenic_values_chunk = await asyncio.gather(*tasks)
    stop_scenic_values += stop_scenic_values_chunk
    end = perf_counter()

    if end - start < 60:
        sleep(60)
    
with open("stop_scenic_values.json", "r") as file:
    original_stop_scenic_values = json.load(file)

with open("stop_scenic_values.json", "w") as file:
    json.dump(original_stop_scenic_values + stop_scenic_values, file)

100%|██████████| 1/1 [01:38<00:00, 98.17s/it]


In [120]:
import json

# Load the JSON file
with open("stop_scenic_values.json", "r") as file:
    data = json.load(file)

# Filter the list based on unique keys
unique_key_elements = {}
for entry in data:
    unique_key_elements.update(entry)  # This ensures only the last occurrence of a key is kept

# Convert the unique_key_elements dictionary back to a list of dictionaries
filtered_data = [{key: value} for key, value in unique_key_elements.items()]

# Save the result back to the file
with open("stop_scenic_values.json", "w") as file:
    json.dump(filtered_data, file, indent=4)

print("Length of filtered list (unique keys):", len(filtered_data))


Length of filtered list (unique keys): 5430
